<a href="https://colab.research.google.com/github/gergogomori/brain_wide_pes/blob/main/pes_sess_to_probes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install ONE-api
! pip install ibllib
! pip install iblatlas


In [ ]:
import numpy as np
import pickle
import shutil

from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org', password='international', silent=True)

from iblatlas.atlas import AllenAtlas
brain_atlas = AllenAtlas()

from brainbox.io.one import SpikeSortingLoader

import warnings
import one.alf.exceptions as alferr
warnings.filterwarnings("ignore", category=alferr.ALFWarning)

# Load EID-s
eids = one.search(datasets='spikes.times.npy')
print('Number of sessions available:', len(eids))


In [ ]:
with open('*** Update the folder path here to match your directory structure ***', 'rb') as f:
    trials_all = pickle.load(f)


In [ ]:
probes_all = {}

for i, EID in enumerate(sorted(list(trials_all.keys()))):

    print('Done with', np.around(i / len(trials_all.keys()) * 100, 1), '% of all sessions.')

    pids, _ = one.eid2pid(EID)

    for PID in pids:

        sl = SpikeSortingLoader(pid=PID, one=one, atlas=brain_atlas)
        spikes, clusters, channels = sl.load_spike_sorting()
        clusters = sl.merge_clusters(spikes, clusters, channels)

        PID = str(PID)

        if clusters is not None:
            for i_cl in range(len(clusters['uuids'])):

                curr_cl = clusters['cluster_id'][i_cl]

                if clusters['label'][i_cl] == 1.0:

                    if EID not in probes_all.keys():
                        probes_all[EID] = {}

                    if PID not in probes_all[EID].keys():
                        probes_all[EID][PID] = [curr_cl]
                    else:
                        probes_all[EID][PID].append(curr_cl)

    shutil.rmtree('/root/Downloads/ONE/openalyx.internationalbrainlab.org/' + one.get_details(EID)['lab'], ignore_errors=True)

with open('*** Update the folder path here to match your directory structure ***', 'wb') as f:
    pickle.dump(probes_all, f, pickle.HIGHEST_PROTOCOL)
